# U/R + Λ Framework Demo

Mach–Zehnder + Bell/CHSH sandbox with a minimal U/R + Λ engine.


## Mach–Zehnder in U/R language

We model a single-photon Mach–Zehnder interferometer as a 2D path space
$\mathcal{H}_U = \mathrm{span}\{|1\rangle, |2\rangle\}$, with a first beam splitter,
a phase shifter $\phi$ in arm 2, and a second beam splitter. Sector R only records
which detector clicked.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# Bare Mach–Zehnder (Sector U)
# -----------------------------

# Beam splitter (50/50)
B = (1/np.sqrt(2)) * np.array([[1, 1],
                               [1, -1]])

def P_phi(phi):
    """Phase shifter: phase on path |2⟩."""
    return np.array([[1, 0],
                     [0, np.exp(1j*phi)]])

# Initial state: photon in input 1
psi_in = np.array([1, 0], dtype=complex)

def mz_output_amplitudes(phi):
    """Apply BS1 → P(phi) → BS2 to psi_in."""
    psi1 = B @ psi_in           # after first BS
    psi2 = P_phi(phi) @ psi1    # after phase shifter
    psi_out = B @ psi2          # after second BS
    return psi_out  # [amp at D0, amp at D1]

phis = np.linspace(0, 2*np.pi, 200)
P_D0 = []
P_D1 = []

for phi in phis:
    amp = mz_output_amplitudes(phi)
    P_D0.append(np.abs(amp[0])**2)
    P_D1.append(np.abs(amp[1])**2)

P_D0 = np.array(P_D0)
P_D1 = np.array(P_D1)

plt.figure(figsize=(6,4))
plt.plot(phis, P_D0, label="P(D0)")
plt.plot(phis, P_D1, label="P(D1)")
plt.xlabel("φ")
plt.ylabel("Probability")
plt.title("Mach–Zehnder output probabilities (Sector U)")
plt.legend()
plt.grid(True)
plt.show()


## Minimal Λ and URExperiment engine

We now introduce a tiny engine layer: a LambdaState container and a URExperiment
class that explicitly separate U-sector evolution, record-making in R, and Λ updates.


In [ ]:
class LambdaState:
    """Minimal Λ container for the U/R framework."""

    def __init__(self, E=1.0, eta=0.0, g=None, A_mean=0.0):
        self.E = E
        self.eta = eta
        self.g = g
        self.A_mean = A_mean

    def collapse_update(self, outcome_label):
        """Placeholder collapse rule for Λ.
        In a full HRZ model this would damp eta, flatten g, etc.
        """
        # Toy example: reset coherence and add a small drift.
        self.E = 1.0
        self.eta *= 0.5
        self.A_mean += 0.01

    def drift(self, dt):
        """Hidden evolution between triggers (placeholder)."""
        # No dynamics yet; hook for future work.
        pass


class URExperiment:
    """Minimal container for a U/R experiment with Λ."""

    def __init__(self, name, H_U_dim, H_R_dim):
        self.name = name
        self.H_U_dim = H_U_dim
        self.H_R_dim = H_R_dim
        self.state_U = None   # column vector in H_U
        self.state_R = None   # column vector in H_R
        self.lambda_state = None  # LambdaState
        self.history = []

    def set_initial_state(self, psi_U, psi_R, lambda_state):
        self.state_U = psi_U
        self.state_R = psi_R
        self.lambda_state = lambda_state

    def apply_unitary(self, U):
        """Apply a unitary on U only: |ψ_U⟩ → U|ψ_U⟩."""
        self.state_U = U @ self.state_U

    def apply_instrument(self, Ms, record_map):
        """Apply a quantum instrument {M_k} on U and update R, Λ.

        Ms: list of Kraus operators on H_U.
        record_map: function (k, state_R, lambda_state) -> (new_R, new_Lambda).
        Returns: outcome index k.
        """
        # Born probabilities
        probs = []
        for Mk in Ms:
            psi_k = Mk @ self.state_U
            pk = np.vdot(psi_k, psi_k).real
            probs.append(pk)
        probs = np.array(probs)
        probs = probs / probs.sum()

        # Sample outcome
        k = np.random.choice(len(Ms), p=probs)

        # Post-measurement U-state
        Mk = Ms[k]
        psi_k = Mk @ self.state_U
        self.state_U = psi_k / np.linalg.norm(psi_k)

        # Update R and Λ
        self.state_R, self.lambda_state = record_map(
            k, self.state_R, self.lambda_state
        )

        self.history.append(("instrument", k, probs.copy()))
        return k


## Mach–Zehnder as a U/R + Λ experiment

We now wrap the Mach–Zehnder setup in a URExperiment object, with an explicit
record basis in R and a toy Λ update at the detector plane.


In [ ]:
def make_mzi_experiment(phi):
    """Build and run a single Mach–Zehnder experiment in U/R form.
    Returns (experiment, outcome k ∈ {0,1}).
    """
    # U: 2D path space; R: three records {0_R, D0, D1}
    exp = URExperiment("MZI", H_U_dim=2, H_R_dim=3)

    psi_U = np.array([1, 0], dtype=complex)          # |1⟩
    psi_R = np.array([1, 0, 0], dtype=complex)       # |0_R⟩
    lam = LambdaState(E=1.0, eta=0.0, g=None, A_mean=0.0)

    exp.set_initial_state(psi_U, psi_R, lam)

    # U-sector optics
    exp.apply_unitary(B)            # BS1
    exp.apply_unitary(P_phi(phi))   # phase
    exp.apply_unitary(B)            # BS2

    # Detector instrument on U
    M0 = np.array([[1, 0],
                   [0, 0]], dtype=complex)
    M1 = np.array([[0, 0],
                   [0, 1]], dtype=complex)
    Ms = [M0, M1]

    def record_map(k, state_R, lambda_state):
        # R basis: |0_R⟩, |D0⟩, |D1⟩
        new_R = np.zeros_like(state_R)
        new_R[k + 1] = 1.0  # k=0 -> |D0⟩, k=1 -> |D1⟩
        lambda_state.collapse_update(k)
        return new_R, lambda_state

    k = exp.apply_instrument(Ms, record_map)
    return exp, k


def run_mzi_trials(phi, N=1000):
    """Monte Carlo estimate of P(D0), P(D1) at phase φ."""
    counts = np.zeros(2, dtype=int)
    for _ in range(N):
        _, k = make_mzi_experiment(phi)
        counts[k] += 1
    return counts / N

# Quick test at φ = π/3
phi_test = np.pi / 3
freqs = run_mzi_trials(phi_test, N=2000)
print("Estimated P(D0), P(D1) at φ = π/3:", freqs)


## Bell/CHSH test in U/R language

We consider a standard Bell pair shared by Alice and Bob. Sector U holds
the two-qubit state and measurement operators; Sector R records their
settings and outcomes. We compute the CHSH value using standard
quantum operators.


In [ ]:
# ----------------------------
# Bell/CHSH in Sector U only
# ----------------------------

# Pauli matrices
sx = np.array([[0, 1],
               [1, 0]], dtype=complex)
sy = np.array([[0, -1j],
               [1j, 0]], dtype=complex)
sz = np.array([[1, 0],
               [0, -1]], dtype=complex)
I2 = np.eye(2, dtype=complex)

def kron(a, b):
    return np.kron(a, b)

# Bell state |Phi+> = (|00> + |11>)/sqrt(2)
phi_plus = (1/np.sqrt(2)) * np.array([1, 0, 0, 1], dtype=complex)

def expectation(state, operator):
    return np.vdot(state, operator @ state).real

# Measurement settings
# Alice: A0 = sz, A1 = sx
# Bob:   B0 = (sz + sx)/sqrt(2), B1 = (sz - sx)/sqrt(2)
A0 = sz
A1 = sx
B0 = (sz + sx) / np.sqrt(2)
B1 = (sz - sx) / np.sqrt(2)

# Correlators E(Ai,Bj)
E00 = expectation(phi_plus, kron(A0, B0))
E01 = expectation(phi_plus, kron(A0, B1))
E10 = expectation(phi_plus, kron(A1, B0))
E11 = expectation(phi_plus, kron(A1, B1))

S = E00 + E01 + E10 - E11
print("E00, E01, E10, E11 =", E00, E01, E10, E11)
print("CHSH S =", S)
